# Start: Analysiere Daten

In [154]:
# import initial_data.csv
import pandas as pd
import numpy as np

from feasibility import is_feasible

# import data from csv file
def import_data():
    data = pd.read_csv('initial_data.csv')
    return data

data = import_data()

In [155]:
# put column names into a list
column_names = data.columns.values
print(column_names)
inputs = column_names[0:8]
print(inputs)
outputs = column_names[8:13]
print(outputs)

['Engine speed' 'Engine load' 'Railpressure' 'Air supply' 'Crank angle'
 'Intake pressure' 'Back pressure' 'Intake temperature' 'NOx' 'PM 1' 'CO2'
 'PM 2' 'Pressure cylinder']
['Engine speed' 'Engine load' 'Railpressure' 'Air supply' 'Crank angle'
 'Intake pressure' 'Back pressure' 'Intake temperature']
['NOx' 'PM 1' 'CO2' 'PM 2' 'Pressure cylinder']


In [156]:
def get_input_data(row):
    x1, x2, x3, x4, x5, x6, x7, x8 = data.iloc[row, 0], data.iloc[row, 1], data.iloc[row, 2], data.iloc[row, 3], data.iloc[row, 4], data.iloc[row, 5], data.iloc[row, 6], data.iloc[row, 7]
    return x1, x2, x3, x4, x5, x6, x7, x8

def get_output_data(row):
    x1, x2, x3, x4, x5 = data.iloc[row, 8], data.iloc[row, 9], data.iloc[row, 10], data.iloc[row, 11], data.iloc[row, 12]
    return x1, x2, x3, x4, x5

def get_critical_output_data(i):
    x1, x2, x3 = data.iloc[i, 9], data.iloc[i, 11], data.iloc[i, 12]
    return x1, x2, x3

In [157]:
# show if data row is feasible via the first 8 columns and the function is_feasible
feasible = []
for i in range(len(data)):
    x1, x2, x3, x4, x5, x6, x7, x8 = get_input_data(i)
    # print(x1, x2, x3, x4, x5, x6, x7, x8)
    feasible.append(is_feasible(x1, x2, x3, x4, x5, x6, x7, x8))
data['feasible'] = feasible

In [158]:
# Check if outputs are in safe range
# PM 1 < 6, PM 2 < 16, Pressure cylinder < 160
safe = []
for i in range(len(data)):
    x9, x10, x11 = get_critical_output_data(i)
    if x9 < 6 and x10 < 16 and x11 < 160:
        safe.append(True)
    else:
        safe.append(False)
data['safe'] = safe

In [159]:
# show unsafe data (safe = False)
unsafe_data = data[data['safe'] == False]
print(unsafe_data)

    Engine speed  Engine load  Railpressure  Air supply  Crank angle  \
70        1800.0        35.66        1521.7    310.5600         5.69   
89        2000.0       139.54        2500.3   1008.3008         3.66   

    Intake pressure  Back pressure  Intake temperature         NOx      PM 1  \
70           1123.5         1236.1                60.9   66.869420  4.095885   
89           3129.9         3329.4                79.7  328.147428  4.017909   

           CO2       PM 2  Pressure cylinder  feasible   safe  
70   36.291467  16.283255          60.497930      True  False  
89  163.080520   4.692198         160.047375      True  False  


In [160]:
# predict when to get safe data and when to get unsafe data
# PM 1 < 6, PM 2 < 16, Pressure cylinder < 160

In [161]:
import csv
import re

def read_csv_data_init(file_path):
    data = []
    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile)
        # Überschriftszeile überspringen
        next(reader, None)
        for row in reader:
            data.append(row)
    return data

def read_csv_data_querry(file_path):
    # Initialisiere eine leere Liste, um die bereinigten Daten zu speichern
    cleaned_data = []

    # Lese die CSV-Datei
    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=';')  # Annahme: Semikolon als Trennzeichen
        for row in reader:
            # Verbinde die Zeilenelemente mit einem Komma, um das Trennzeichen zu vereinheitlichen
            unified_row = ','.join(row)
            # Ersetze mehrere aufeinander folgende Kommas durch ein einzelnes Komma
            unified_row = re.sub(r',+', ',', unified_row)
            # Teile die vereinheitlichte Zeile nach dem Komma auf
            split_row = unified_row.split(',')
            # Entferne die ersten zwei Parameter
            cleaned_row = split_row[1:]
            # Füge die bereinigte Zeile der Liste hinzu
            cleaned_data.append(cleaned_row)
    return cleaned_data[1:]
    #return cleaned_data


# Beispiel: Daten aus "input.csv" einlesen
input_file_path_querry = "querys_ForcePush (1).csv"
input_file_path_init = "initial_data.csv"

querryData = read_csv_data_querry(input_file_path_querry)

initData = read_csv_data_init(input_file_path_init)


#data = initData + querryData
data = initData


In [162]:
i_x = [row[:8] for row in data]
i_x = pd.DataFrame(i_x)
i_x = i_x.astype(float)

i_y = [row[-5:] for row in data]
i_y = pd.DataFrame(i_y)
i_y = i_y.astype(float)

init_var = i_y.std()
init_mean = i_y.mean()

In [163]:
data = read_csv_data_init("submission.csv")

s_x = [row[:8] for row in data]
s_x = pd.DataFrame(s_x)
s_x = s_x.astype(float)

In [178]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

x = i_x
y = i_y

# Daten in Trainings- und Testdatensatz aufteilen 
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Hyperparameter-Tuning mit Grid-Suche für Random Forest Regressor 
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'] 
}

rf_model = RandomForestRegressor(random_state=42)

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error') 
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print(f"Beste Hyperparameter: {best_params}")

# Modell mit den besten Hyperparametern trainieren 
best_rf_model = RandomForestRegressor(**best_params, random_state=42) 
best_rf_model.fit(X_train, y_train)

# Vorhersagen machen
y_pred_rf = best_rf_model.predict(X_test)

# Modellbewertung für Random Forest
mse_rf = mean_squared_error(np.array(y_test), y_pred_rf) 
print(f"Mean Squared Error (Random Forest): {mse_rf}") 
mape_rf = mean_absolute_percentage_error(np.array(y_test), y_pred_rf) 
print(f"Mean Absolute Percentage Error (Random Forest): {mape_rf}")

# MAPE für jeden Ausgabeparameter nach Random Forest 
mape_per_output_rf = mean_absolute_percentage_error(np.array(y_test), y_pred_rf, multioutput='raw_values') 
for i, mape in enumerate(mape_per_output_rf):
    print(f"MAPE für Ausgabeparameter {i+1} (Random Forest): {mape}")


In [177]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error


# Daten in Trainings- und Testdatensatz aufteilen
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# y_test = np.array(y_test)

# Decision Tree Regressor Modell erstellen und trainieren
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

# Vorhersagen machen
y_pred = model.predict(X_test)

# Modellbewertung ohne Pruning


# Pruning mit Grid-Suche für Hyperparameter-Optimierung
param_grid = {
    'max_depth': [1, 3, 5, 7, 10, 13],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': ['sqrt', 'log2']
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_

# Modell mit den besten Hyperparametern trainieren
best_model = DecisionTreeRegressor(**best_params)
best_model.fit(X_train, y_train)

# Vorhersagen machen mit dem geprüften Modell
y_pred_pruned = best_model.predict(X_test)

# Modellbewertung nach Pruning


# Random Forest Regressor Modell erstellen und trainieren
rf_model = RandomForestRegressor(n_estimators=100)
rf_model.fit(X_train, y_train)

# Vorhersagen machen mit dem Random Forest Modell
y_pred_rf = rf_model.predict(X_test)

# Modellbewertung für Random Forest


print(f"init var: {init_var}")
print(f"init mean: {init_mean}")
print("\n")

model_y = model.predict(s_x)
model_y = pd.DataFrame(model_y)
model_y = model_y.astype(float)
print(f"Quotienten model var: {(model_y.std())/init_var}")
print(f"Quotienten model mean: {(model_y.mean())/init_mean}")
print("\n")

best_model = model.predict(s_x)
best_model = pd.DataFrame(best_model)
best_model = best_model.astype(float)
print(f"Quotienten model var: {(best_model.std())/init_var}")
print(f"Quotienten model mean: {best_model.mean()/init_mean}")
print("\n")

rf_model = rf_model.predict(s_x)
rf_model = pd.DataFrame(rf_model)
rf_model = rf_model.astype(float)
print(f"Quotienten model var: {(rf_model.std())/init_var}")
print(f"Quotienten model mean: {(rf_model.mean())/init_mean}")



init var: 0    134.135904
1      1.305993
2     40.338013
3      3.059437
4     28.450298
dtype: float64
init mean: 0    170.290383
1      1.901737
2     59.829099
3      5.450735
4     82.803179
dtype: float64


Quotienten model var: 0    0.948146
1    0.800914
2    0.813154
3    0.893079
4    0.832030
dtype: float64
Quotienten model mean: 0    0.972940
1    0.741922
2    0.823625
3    0.857070
4    0.928192
dtype: float64


Quotienten model var: 0    0.948146
1    0.800914
2    0.813154
3    0.893079
4    0.832030
dtype: float64
Quotienten model mean: 0    0.972940
1    0.741922
2    0.823625
3    0.857070
4    0.928192
dtype: float64


Quotienten model var: 0    0.858981
1    0.578753
2    0.725920
3    0.474298
4    0.755278
dtype: float64
Quotienten model mean: 0    0.962029
1    0.935452
2    0.937518
3    0.945419
4    0.960312
dtype: float64


In [176]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error


# Daten in Trainings- und Testdatensatz aufteilen
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# y_test = np.array(y_test)

# Decision Tree Regressor Modell erstellen und trainieren
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

# Vorhersagen machen
y_pred = model.predict(X_test)

# Modellbewertung ohne Pruning
mse_before_pruning = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (vor Pruning): {mse_before_pruning}")
mape_before_pruning = mean_absolute_percentage_error(y_test, y_pred=y_pred, multioutput='raw_values')*100

mse_per_output = mean_squared_error(y_test, y_pred, multioutput='raw_values')
for i, mse in enumerate(mse_per_output):
    print(f"Mean Squared Error für Ausgabeparameter {i+1}: {mse} : {mse/var[i]}")
for i, mse in enumerate(mape_before_pruning):
    print(f"Mean Absolut Percantage Error für Ausgabeparameter {i+1}: {mse}")

# Pruning mit Grid-Suche für Hyperparameter-Optimierung
param_grid = {
    'max_depth': [1, 3, 5, 7, 10, 13],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': ['auto', 'sqrt', 'log2']
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_

# Modell mit den besten Hyperparametern trainieren
best_model = DecisionTreeRegressor(**best_params)
best_model.fit(X_train, y_train)

# Vorhersagen machen mit dem geprüften Modell
y_pred_pruned = best_model.predict(X_test)

# Modellbewertung nach Pruning
mse_after_pruning = mean_squared_error(y_test, y_pred_pruned)
print(f"Mean Squared Error (nach Pruning): {mse_after_pruning}")
mape_after_pruning = mean_absolute_percentage_error(y_test, y_pred=y_pred_pruned, multioutput='raw_values')*100

mse_per_output = mean_squared_error(y_test, y_pred_pruned, multioutput='raw_values')
for i, mse in enumerate(mse_per_output):
    print(f"Mean Squared Error für Ausgabeparameter {i+1}: {mse} : {mse/var[i]}")
for i, mse in enumerate(mape_after_pruning):
    print(f"Mean Absolut Percantage Error für Ausgabeparameter {i+1}: {mse}")

# Random Forest Regressor Modell erstellen und trainieren
rf_model = RandomForestRegressor(n_estimators=100)
rf_model.fit(X_train, y_train)

# Vorhersagen machen mit dem Random Forest Modell
y_pred_rf = rf_model.predict(X_test)

# Modellbewertung für Random Forest
mse_rf = mean_squared_error(y_test, y_pred_rf)
print(f"Mean Squared Error (Random Forest): {mse_rf}")
mape_rf = mean_absolute_percentage_error(y_test, y_pred=y_pred_rf, multioutput='raw_values')*100

mse_per_output = mean_squared_error(y_test, y_pred_rf, multioutput='raw_values')
for i, mse in enumerate(mse_per_output):
    print(f"Mean Squared Error für Ausgabeparameter {i+1}: {mse} : {mse/var[i]}")
for i, mse in enumerate(mape_rf):
    print(f"Mean Absolut Percantage Error für Ausgabeparameter {i+1}: {mse}")


# new_Input = [[1216.320277750492, 169.48516483418643, 1501.1897808736858, 555.3636147531786, 6.218004648108035, 2903.9827555595525, 2098.829896093796, 56.01301888900216]]
# predict_output = best_model.predict(new_Input)
# print(f"Ewarteter Ausgabewert für neue Eingabe (nach Pruning): {predict_output}")

# predict_output = rf_model.predict(new_Input)
# print(f"Ewarteter Ausgabewert für neue Eingabe (Random Forest): {predict_output}")

Mean Squared Error (vor Pruning): 256.9242345774802
Mean Squared Error für Ausgabeparameter 1: 1189.6459491152416 : 0.066119208895669
Mean Squared Error für Ausgabeparameter 2: 0.4802440599498167 : 0.2815661204604644
Mean Squared Error für Ausgabeparameter 3: 41.834136507145686 : 0.025709983474385
Mean Squared Error für Ausgabeparameter 4: 9.982944895931851 : 1.0665360067472907
Mean Squared Error für Ausgabeparameter 5: 42.6778983091318 : 0.05272655238017956
Mean Absolut Percantage Error für Ausgabeparameter 1: 18.797339200283066
Mean Absolut Percantage Error für Ausgabeparameter 2: 30.69505308888721
Mean Absolut Percantage Error für Ausgabeparameter 3: 24.01394207237853
Mean Absolut Percantage Error für Ausgabeparameter 4: 36.62418018477196
Mean Absolut Percantage Error für Ausgabeparameter 5: 6.80344932511892
Mean Squared Error (nach Pruning): 816.3694253411219
Mean Squared Error für Ausgabeparameter 1: 3889.4902549236967 : 0.21617357572160875
Mean Squared Error für Ausgabeparameter 

c:\Users\CM3EXN\dev\force-push\myenv\lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
480 fits failed out of a total of 1440.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
480 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\CM3EXN\dev\force-push\myenv\lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\CM3EXN\dev\force-push\myenv\lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\Users\CM3EXN\dev\force-push\myenv\lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(